<a href="https://colab.research.google.com/github/eddycc66/inundaci-n/blob/master/simulacioninudacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas rasterio imageio

In [ ]:
!unzip -q datos.zip -d datos

In [1]:
# Autor M.Sc. Edwin Calle Condori
# ANÁLISIS DE INUNDACIÓN
import os
import numpy as np
import rasterio
import geopandas as gpd
import pandas as pd
from rasterio import features
import matplotlib.pyplot as plt
import imageio

# --- FUNCIONES AUXILIARES ---
def calcular_hillshade(dem, azimuth=315, altitude=45):
    x, y = np.gradient(dem)
    slope = np.pi/2 - np.arctan(np.sqrt(x**2 + y**2))
    aspect = np.arctan2(-x, y)
    azimuth_rad = np.radians(azimuth)
    altitude_rad = np.radians(altitude)

    shaded = (np.sin(altitude_rad) * np.sin(slope) +
              np.cos(altitude_rad) * np.cos(slope) *
              np.cos(azimuth_rad - aspect))

    return (shaded * 255).astype(np.uint8)

def propagar_inundacion(dem, semillas, nivel_agua, pasos=50):
    inundacion = np.zeros_like(dem, dtype=bool)
    y_seed, x_seed = np.where(semillas)
    puntos_activos = list(zip(y_seed, x_seed))

    for _ in range(pasos):
        nuevos_puntos = []
        for y, x in puntos_activos:
            elev_actual = dem[y, x] + nivel_agua

            for dy in [-1, 0, 1]:
                for dx in [-1, 0, 1]:
                    if dy == 0 and dx == 0:
                        continue

                    ny, nx = y + dy, x + dx

                    if 0 <= ny < dem.shape[0] and 0 <= nx < dem.shape[1]:
                        condicion = (
                            not inundacion[ny, nx] and
                            dem[ny, nx] <= elev_actual and
                            dem[ny, nx] >= dem[y, x] - 0.5
                        )

                        if condicion:
                            inundacion[ny, nx] = True
                            nuevos_puntos.append((ny, nx))

        puntos_activos = nuevos_puntos
        if not puntos_activos:
            break

    return inundacion.astype(np.uint8)

# --- CONFIGURACIÓN DE RUTAS PARA COLAB ---
ROOT_DIR = "/content/datos"
INPUT_DIR = os.path.join(ROOT_DIR, "datos")
OUTPUT_DIR = os.path.join(ROOT_DIR, "resultados")

# Crear carpetas de resultados
os.makedirs(OUTPUT_DIR, exist_ok=True)
for subdir in ['rasters', 'vectores', 'logs', 'animacion']:
    os.makedirs(os.path.join(OUTPUT_DIR, subdir), exist_ok=True)

# --- CARGA DE DATOS ---
with rasterio.open(os.path.join(INPUT_DIR, "raster", "dem.tif")) as src:
    dem = src.read(1)
    meta = src.meta.copy()
    transform = src.transform
    crs = src.crs

with rasterio.open(os.path.join(INPUT_DIR, "raster", "cuenca.tif")) as src:
    cuenca = src.read(1)
    cuenca = np.where(cuenca > 0, 1, 0)

gdf_rio = gpd.read_file(os.path.join(INPUT_DIR, "vector", "rio.shp")).to_crs(crs)
rio_mask = features.rasterize(
    [(geom, 1) for geom in gdf_rio.geometry],
    out_shape=dem.shape,
    transform=transform
)

# --- PROCESAMIENTO PRINCIPAL ---
hillshade = calcular_hillshade(dem)
meta_hillshade = meta.copy()
meta_hillshade.update({'dtype': 'uint8', 'nodata': 0})

with rasterio.open(os.path.join(OUTPUT_DIR, 'rasters', 'hillshade.tif'), 'w', **meta_hillshade) as dst:
    dst.write(hillshade, 1)

niveles_agua = np.arange(0, 2, 0.2)
log_data = []
meta_inundacion = meta.copy()
meta_inundacion.update({'dtype': 'uint8', 'nodata': 0})

for i, nivel in enumerate(niveles_agua):
    semillas = (rio_mask == 1) & (dem <= (dem + nivel))
    inundacion = propagar_inundacion(dem, semillas, nivel)
    inundacion = inundacion * cuenca

    with rasterio.open(os.path.join(OUTPUT_DIR, 'rasters', f'inundacion_{nivel:.1f}m.tif'), 'w', **meta_inundacion) as dst:
        dst.write(inundacion, 1)

    area_m2 = np.sum(inundacion) * abs(transform[0])**2
    log_data.append([nivel, area_m2, area_m2/1e6, f'inundacion_{nivel:.1f}m.tif'])

    fig, ax = plt.subplots(figsize=(12, 8))
    ax.imshow(hillshade, cmap='gray', alpha=0.4)
    ax.imshow(dem, cmap='terrain', alpha=0.6)
    ax.imshow(inundacion, cmap='Blues', alpha=0.5)
    ax.set_title(f'Nivel: {nivel:.1f}m | Área: {area_m2/1e6:.2f} km²')
    ax.axis('off')

    plt.savefig(os.path.join(OUTPUT_DIR, 'animacion', f'frame_{i:03d}.png'), dpi=150, bbox_inches='tight')
    plt.close()

df_log = pd.DataFrame(log_data, columns=['Nivel_agua(m)', 'Area_m2', 'Area_km2', 'Archivo'])
df_log.to_csv(os.path.join(OUTPUT_DIR, 'logs', 'log_inundacion.csv'), index=False)

archivos_inundacion = [f for f in os.listdir(os.path.join(OUTPUT_DIR, 'rasters')) if f.startswith('inundacion_')]
inundacion_max = np.max([rasterio.open(os.path.join(OUTPUT_DIR, 'rasters', f)).read(1) for f in archivos_inundacion], axis=0)

with rasterio.open(os.path.join(OUTPUT_DIR, 'rasters', 'inundacion_maxima.tif'), 'w', **meta_inundacion) as dst:
    dst.write(inundacion_max, 1)

archivos_animacion = [os.path.join(OUTPUT_DIR, 'animacion', f'frame_{i:03d}.png') for i in range(len(niveles_agua))]

with imageio.get_writer(os.path.join(OUTPUT_DIR, 'inundacion_animacion.gif'),
                        mode='I', duration=300, loop=0) as writer:
    for archivo in archivos_animacion:
        image = imageio.v2.imread(archivo)
        writer.append_data(image)

print("✅ Proceso completado exitosamente!")
print(f"📂 Resultados guardados en: {OUTPUT_DIR}")

ModuleNotFoundError: No module named 'rasterio'